In [20]:
import random
from datetime import timedelta
from decimal import Decimal
from faker import Faker
import psycopg2

fake = Faker()

# -------------------------------
# CONFIGURACIÓN
# -------------------------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "testDB",
    "user": "test",
    "password": "test"
}

NUM_USERS = 20
NUM_PRODUCTS = 30
NUM_TICKETS = 100
MAX_ITEMS_PER_TICKET = 5

# -------------------------------
# CONEXIÓN
# -------------------------------
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()



In [6]:
# -------------------------------
# USERS
# -------------------------------
user_ids = []
for _ in range(NUM_USERS):
    cur.execute("""
        INSERT INTO users (name, email)
        VALUES (%s, %s)
        RETURNING id
    """, (fake.name(), fake.unique.email()))
    user_ids.append(cur.fetchone()[0])

conn.commit()



In [12]:
# -------------------------------
# PRODUCTS
# -------------------------------
product_ids = []
for _ in range(NUM_PRODUCTS):
    cur.execute("""
        INSERT INTO products (sku, name, description, status)
        VALUES (%s, %s, %s, 'ACTIVE')
        RETURNING id
    """, (
        fake.unique.bothify(text="SKU-####"),
        fake.word().capitalize(),
        fake.sentence()
    ))
    product_ids.append(cur.fetchone()[0])

conn.commit()



In [21]:
# -------------------------------
# PRODUCT PRICES (HISTÓRICO CORRECTO)
# -------------------------------
for product_id in product_ids:
    versions = random.randint(1, 4)

    # Fecha inicial
    valid_from = fake.date_time_between(start_date="-2y", end_date="-1y")

    prices = []

    for i in range(versions):
        price = Decimal(str(round(random.uniform(5, 500), 2)))
        next_date = valid_from + timedelta(days=random.randint(30, 180))

        prices.append({
            "price": price,
            "valid_from": valid_from,
            "valid_to": None  # se ajusta luego
        })

        valid_from = next_date

    # Marcar todos menos el último como inactivos
    for i in range(len(prices) - 1):
        prices[i]["valid_to"] = prices[i + 1]["valid_from"]

    # Insertar en BD
    for p in prices:
        cur.execute("""
            INSERT INTO product_prices (
                product_id,
                price,
                valid_from,
                valid_to
            )
            VALUES (%s, %s, %s, %s)
        """, (
            product_id,
            p["price"],
            p["valid_from"],
            p["valid_to"]
        ))

conn.commit()

In [22]:
conn.commit()

In [18]:

# -------------------------------
# HELPER: PRECIO VIGENTE
# -------------------------------
def get_active_price(product_id):
    cur.execute("""
        SELECT price
        FROM product_prices
        WHERE product_id = %s
          AND valid_to IS NULL
    """, (product_id,))
    return cur.fetchone()[0]

# -------------------------------
# TICKETS + ITEMS
# -------------------------------
for _ in range(NUM_TICKETS):
    user_id = random.choice(user_ids)
    ticket_date = fake.date_time_between(start_date="-6m", end_date="now")

    cur.execute("""
        INSERT INTO tickets (user_id, total_amount, created_at)
        VALUES (%s, 0, %s)
        RETURNING id
    """, (user_id, ticket_date))

    ticket_id = cur.fetchone()[0]
    total = Decimal("0.00")

    for _ in range(random.randint(1, MAX_ITEMS_PER_TICKET)):
        product_id = random.choice(product_ids)
        quantity = random.randint(1, 5)
        unit_price = get_active_price(product_id)
        subtotal = unit_price * quantity
        total += subtotal

        cur.execute("""
            INSERT INTO ticket_items (
                ticket_id,
                product_id,
                quantity,
                unit_price,
                subtotal
            )
            VALUES (%s, %s, %s, %s, %s)
        """, (
            ticket_id,
            product_id,
            quantity,
            unit_price,
            subtotal
        ))

    cur.execute("""
        UPDATE tickets
        SET total_amount = %s
        WHERE id = %s
    """, (total, ticket_id))

conn.commit()

# -------------------------------
# CLEANUP
# -------------------------------
cur.close()
conn.close()

print("✅ Datos aleatorios insertados correctamente")

✅ Datos aleatorios insertados correctamente
